In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.options import DesiredCapabilities
from selenium.webdriver.common.action_chains import ActionChains
from calendar import monthrange
from datetime import datetime
import numpy as np

import requests
from bs4 import BeautifulSoup as bs


In [21]:
#browser/setup
from selenium import webdriver
def chrome(headless=False, page_load_timeout=10, implicit_wait_time=2):
    # Настройки для Chrome WebDriver
    geckodriver_path='geckodriver.exe'
    firefox_options = webdriver.FirefoxOptions()
    if headless:
        firefox_options.add_argument("--headless")
    
    # Создаем экземпляр WebDriver Firefox с явно указанным путем к geckodriver
    browser = webdriver.Firefox(options=firefox_options)
    browser.implicitly_wait(implicit_wait_time)
    browser.set_page_load_timeout(page_load_timeout)
    
    return browser

    

In [22]:
new_link  = "https://store.steampowered.com/search/?term="
browser= chrome(0,12,20)
browser.maximize_window()
browser.get(new_link)
games_links = []
try:
    i = 1
    while True:
        time.sleep(0.25)
        if i == 2: # сюда колво данных на  i приходится 10 игр по стандарту открыто 50 при i=0
            break
        actions = ActionChains(browser)
        actions.send_keys(Keys.PAGE_DOWN)
        actions.perform()
        i+=1
    elem = browser.find_elements(By.XPATH, '//*[@id="search_resultsRows"]/a')
    game_links = [(a.get_property("href"),a.get_attribute("data-ds-appid")) for a in elem]
    browser.quit() 
    for el in game_links:
        time.sleep(0.1)
        print(el)
except Exception as e:
        print(e)


('https://store.steampowered.com/app/730/CounterStrike_2/?snr=1_7_7_230_150_1', '730')
('https://store.steampowered.com/app/252490/Rust/?snr=1_7_7_230_150_1', '252490')
('https://store.steampowered.com/app/1086940/Baldurs_Gate_3/?snr=1_7_7_230_150_1', '1086940')
('https://store.steampowered.com/app/227300/Euro_Truck_Simulator_2/?snr=1_7_7_230_150_1', '227300')
('https://store.steampowered.com/app/578080/PUBG_BATTLEGROUNDS/?snr=1_7_7_230_150_1', '578080')
('https://store.steampowered.com/app/594650/Hunt_Showdown/?snr=1_7_7_230_150_1', '594650')
('https://store.steampowered.com/app/648800/Raft/?snr=1_7_7_230_150_1', '648800')
('https://store.steampowered.com/app/1282100/Remnant_II/?snr=1_7_7_230_150_1', '1282100')
('https://store.steampowered.com/app/1326470/Sons_Of_The_Forest/?snr=1_7_7_230_150_1', '1326470')
('https://store.steampowered.com/app/570/Dota_2/?snr=1_7_7_230_150_1', '570')
('https://store.steampowered.com/app/394360/Hearts_of_Iron_IV/?snr=1_7_7_230_150_1', '394360')
('https

In [4]:
datadict={
        "name":[],
        "first_30d":[],
        "first_90d":[],
        "price":[],
        "developer":[],
        "publisher":[],
        "genres":[],
        "steam_release_date":[],
        "app_id":[],
        "details":[],
        "series":[],
        "language":[]
        }    

In [5]:
err = 0
print(time.asctime())
for link in game_links:
    #подготовка поолей на случай ошибок
    name = None
    price = None
    developer = None
    publisher = None
    genres = None
    steam_release_date = None
    game_details = None
    gets_err = False
    series = False
    lang_dict = dict()
    
    try:
        soup = bs(requests.get(link[0]).text, 'lxml')
        block = soup.find("div",class_="details_block")
        devblock = block.find_all("div",class_="dev_row")
        block = block.text.split()
        
        dev = [dev.text.strip().split(":\n") for dev in devblock]
        developer = dev[0][1]
        publisher = dev[1][1]
        if len(dev) == 3:
            series = dev[2][1]
        
        name = " ".join(block[block.index("Title:")+1:block.index("Genre:")])
        genres = block[block.index("Genre:")+1:block.index("Developer:")]
        di=block.index("Date:")+1
        steam_release_date = " ".join(block[di:di+3])
        game_details = [game.text.strip() for game in soup.find_all("a",class_="game_area_details_specs_ctn")]
        
        lang_block = soup.find("table",class_="game_language_options")
        languages = lang_block.find_all("tr")
        languages = [tr.text.split() for tr in languages]
        languages.pop(0)
        for lang in languages:
            lanc = lang.count("✔")
            my_lang = " ".join(lang[:-lanc])
            lang_dict[my_lang] = lanc
            
        try:
            price = soup.find("div",class_="game_purchase_price price").text.strip()
        except:
            price = soup.find("div",class_="discount_original_price").text.strip()
    
    except Exception as e:
        err +=1
        gets_err = True
    
    finally:
        if gets_err == False:
            datadict["name"].append(name)
            datadict["price"].append(price)
            datadict["developer"].append(developer)
            datadict["publisher"].append(publisher)
            datadict["series"].append(series)
            datadict["genres"].append(genres)
            datadict["steam_release_date"].append(steam_release_date)
            datadict["app_id"].append(link[1])
            datadict["details"].append(game_details)
            datadict["language"].append(lang_dict)
        gets_err = False
        
print(time.asctime())
print(f"Возникло {err} ошибок")
print(datadict)


Mon Mar 20 10:36:51 2023
Mon Mar 20 10:37:15 2023
Возникло 0 ошибок
{'name': ['ギルド受付嬢フィリス', 'Simple Arcade', 'Those Left Behind', 'Biz Builder Delux', "Fisher's Qualm", 'Horny Hotel Maids', 'ANETHA', 'Aquascaping Designer', 'Eternal Hunt', 'Kriaturaz - O Guardião das Lendas (Base)', 'Goospace', 'The Hero Journey in Yggdrasil', 'My Landlord is a Futanari', 'I was born poor', 'Demon Hentai Slayer', 'Goblins', 'Shiner', 'Destiny Duel', 'Eftu', 'Shepherd', 'Battle Tanks: Arena', 'Desolation', 'Lost Dream: Darkness', 'Pato Martinez', 'Vacuum', 'Quantum Gravity', 'Flame Keeper', 'GunnRunner', 'Anthology of Fear', 'Castrum Novum VR - Life in a Roman colony', 'Toy Tactics', 'Flea the Cat', 'MDF: Magical Defense Force', 'Escape From The Depth', '世界征服者-成吉思汗（World Conqueror - Genghis Khan）', 'Souls of Tartarus', 'Running Riot', "Mr. Hopp's Playhouse 3", 'Film Studio Manager', 'Off The Tracks', '十月怪谈-Insane Rules', 'Pleh!', 'CHI NINJA', 'MeiQi:Phoenix and the Chamber of Secrets', 'Parametric Engin

In [6]:
def timef(name):
    dd = {"Jan":1,"Feb":2,"Mar":3,"Apr":4,"May":5,"Jun":6,"Jul":7,"Aug":8,"Sep":9,"Oct":10,"Nov":11,"Dec":12}
    return dd[name]

In [7]:
print(time.asctime())
datadict["first_30d"]= []
datadict["first_90d"] =[]
dateTime = {}
list_of_breakgames = []
def get_secs(date):
    global dateTime
    try:
        if type(date) is str:
            date = date.split()
            date[0] = date[0][:3]
        to_struc = time.strptime(str(timef(date[0])) + ","+  str(date[1]),'%m,%Y')
        if to_struc not in dateTime:
                dateTime[to_struc] = time.mktime(to_struc)
        return dateTime[to_struc]
    except Exception as e:
        return -1
    
    
for iid in range(len(datadict["app_id"])):
    people30 = None
    people90 = None
    try:
        link = "https://steamcharts.com/app/" + datadict["app_id"][iid] + "/"
        br = requests.get(link)
        soup = bs(br.text, 'lxml')
        name_ = [i.text.strip() for i in soup.find_all("td",class_="month-cell")]
        if not soup.find("td",class_="right num-f italic"):
            continue
        avg_pep = [soup.find("td",class_="right num-f italic").text.strip()] + [i.text.strip() for i in soup.find_all("td",class_="right num-f")]
        name_[0] = time.time()
        date = datadict["steam_release_date"][iid].replace(",","").split()
        days = monthrange(int(date[2]), timef(date[1]))[1] - int(date[0])
        name_[1:] = [get_secs(name) for name in name_[1:]] 
        create_secs = get_secs(date[1:])
        my_list_ofdates = []
        i = 0
        while True and i < len(name_):
            if create_secs <= name_[i]:
                i += 1
            else:break
        avg_pep = avg_pep[:i]
        if days > 14 or i==1:
            people30 = avg_pep[-1]
            if i>=2:
                people90 = avg_pep[-3]
        else:
            people30 = avg_pep[-2]
            if i>=3:
                people90 = avg_pep[-4]
    except Exception as e:
        list_of_breakgames.append(link)
    finally:
        datadict["first_30d"].append(people30)
        datadict["first_90d"].append(people90)
print(time.asctime())
list_of_breakgames
datadict

Mon Mar 20 10:37:15 2023
Mon Mar 20 10:37:37 2023


{'name': ['ギルド受付嬢フィリス',
  'Simple Arcade',
  'Those Left Behind',
  'Biz Builder Delux',
  "Fisher's Qualm",
  'Horny Hotel Maids',
  'ANETHA',
  'Aquascaping Designer',
  'Eternal Hunt',
  'Kriaturaz - O Guardião das Lendas (Base)',
  'Goospace',
  'The Hero Journey in Yggdrasil',
  'My Landlord is a Futanari',
  'I was born poor',
  'Demon Hentai Slayer',
  'Goblins',
  'Shiner',
  'Destiny Duel',
  'Eftu',
  'Shepherd',
  'Battle Tanks: Arena',
  'Desolation',
  'Lost Dream: Darkness',
  'Pato Martinez',
  'Vacuum',
  'Quantum Gravity',
  'Flame Keeper',
  'GunnRunner',
  'Anthology of Fear',
  'Castrum Novum VR - Life in a Roman colony',
  'Toy Tactics',
  'Flea the Cat',
  'MDF: Magical Defense Force',
  'Escape From The Depth',
  '世界征服者-成吉思汗（World Conqueror - Genghis Khan）',
  'Souls of Tartarus',
  'Running Riot',
  "Mr. Hopp's Playhouse 3",
  'Film Studio Manager',
  'Off The Tracks',
  '十月怪谈-Insane Rules',
  'Pleh!',
  'CHI NINJA',
  'MeiQi:Phoenix and the Chamber of Secrets',

In [8]:
df = pd.DataFrame(datadict)
df.loc[df["first_30d"].apply(lambda x : x is None) == True]

,name,first_30d,first_90d,price,developer,publisher,genres,steam_release_date,app_id,details,series,language
0,ギルド受付嬢フィリス,None,None,726 pуб.,NAGINATA SOFT,WASABI entertainment,"[Indie,, RPG]","20 Mar, 2023",1989840,[Single-player],WASABI entertainment,"{'English': 2, 'Japanese': 2, 'Simplified Chin..."
1,Simple Arcade,None,None,69 pуб.,Sarni Studio,Sarni Studio,"[Casual,, Indie]","20 Mar, 2023",2333840,[Single-player],Simple Arcade,{'English': 3}
2,Those Left Behind,None,None,520 pуб.,ZDevStudio,ZDevStudio,"[Action,, Indie]","19 Mar, 2023",2101070,[Single-player],False,{'English': 1}
3,Biz Builder Delux,None,None,205 pуб.,"Kairosoft Co.,Ltd","Kairosoft Co.,Ltd","[Casual,, Indie,, Simulation,, Strategy]","19 Mar, 2023",2102480,"[Single-player, Steam Achievements, Full contr...","Kairosoft Co.,Ltd","{'English': 1, 'Simplified Chinese': 1, 'Tradi..."
5,Horny Hotel Maids,None,None,52 pуб.,Horny69Team,Horny69Team,"[Casual,, Indie]","19 Mar, 2023",2271170,[Single-player],False,{'English': 2}
6,ANETHA,None,None,Free To Play,Team Panopticon,"Team Panopticon, Sejong Univ. PandoraCube","[Adventure,, Free, to, Play,, Indie]","19 Mar, 2023",1923800,[Single-player],False,"{'': 0, 'Korean': 1}"
8,Eternal Hunt,None,None,280 pуб.,Lucas Amorim,Lucas Amorim,"[Action,, Adventure,, Indie,, RPG]","18 Mar, 2023",2306680,[Single-player],False,{'English': 3}
9,Kriaturaz - O Guardião das Lendas (Base),None,None,30 pуб.,Messier Games & Animations,Messier Games & Animations,"[Adventure,, Casual,, Free, to, Play,, Indie]","18 Mar, 2023",544690,[Single-player],Universe,"{'': 0, 'Portuguese - Brazil': 2}"
10,Goospace,None,None,165 pуб.,"TTeokLeaf, Wojtek",TTeokLeaf,"[Action,, Adventure,, Casual,, Indie]","18 Mar, 2023",2326190,"[Single-player, Steam Achievements, Steam Lead...",False,"{'English': 1, 'French': 1, 'Italian': 1, 'Ger..."
11,The Hero Journey in Yggdrasil,None,None,Free To Play,"VirtualRehabArt4Health, Hjälteskolan, Kreativi...",VirtualRehabArt4Health,"[Adventure,, Casual,, Free, to, Play,, Indie,,...","18 Mar, 2023",1630440,"[Single-player, VR Only]",False,{'English': 2}


In [9]:
df

,name,first_30d,first_90d,price,developer,publisher,genres,steam_release_date,app_id,details,series,language
0,ギルド受付嬢フィリス,None,None,726 pуб.,NAGINATA SOFT,WASABI entertainment,"[Indie,, RPG]","20 Mar, 2023",1989840,[Single-player],WASABI entertainment,"{'English': 2, 'Japanese': 2, 'Simplified Chin..."
1,Simple Arcade,None,None,69 pуб.,Sarni Studio,Sarni Studio,"[Casual,, Indie]","20 Mar, 2023",2333840,[Single-player],Simple Arcade,{'English': 3}
2,Those Left Behind,None,None,520 pуб.,ZDevStudio,ZDevStudio,"[Action,, Indie]","19 Mar, 2023",2101070,[Single-player],False,{'English': 1}
3,Biz Builder Delux,None,None,205 pуб.,"Kairosoft Co.,Ltd","Kairosoft Co.,Ltd","[Casual,, Indie,, Simulation,, Strategy]","19 Mar, 2023",2102480,"[Single-player, Steam Achievements, Full contr...","Kairosoft Co.,Ltd","{'English': 1, 'Simplified Chinese': 1, 'Tradi..."
4,Fisher's Qualm,0.00,None,125 pуб.,Solder Spark Studios,Solder Spark Studios,"[Casual,, Indie]","19 Mar, 2023",2150460,[Single-player],False,{'English': 3}
5,Horny Hotel Maids,None,None,52 pуб.,Horny69Team,Horny69Team,"[Casual,, Indie]","19 Mar, 2023",2271170,[Single-player],False,{'English': 2}
6,ANETHA,None,None,Free To Play,Team Panopticon,"Team Panopticon, Sejong Univ. PandoraCube","[Adventure,, Free, to, Play,, Indie]","19 Mar, 2023",1923800,[Single-player],False,"{'': 0, 'Korean': 1}"
7,Aquascaping Designer,0.00,None,385 pуб.,Nipinfo,Nipinfo,"[Indie,, Simulation,, Early, Access]","19 Mar, 2023",2242370,[Single-player],False,"{'English': 1, 'French': 1, 'Italian': 1, 'Ger..."
8,Eternal Hunt,None,None,280 pуб.,Lucas Amorim,Lucas Amorim,"[Action,, Adventure,, Indie,, RPG]","18 Mar, 2023",2306680,[Single-player],False,{'English': 3}
9,Kriaturaz - O Guardião das Lendas (Base),None,None,30 pуб.,Messier Games & Animations,Messier Games & Animations,"[Adventure,, Casual,, Free, to, Play,, Indie]","18 Mar, 2023",544690,[Single-player],Universe,"{'': 0, 'Portuguese - Brazil': 2}"


In [10]:
df.to_csv('games10kcool.csv')